# --------------------------------------------------------------------------------------------------------------
# Playas de ESPAÑA 2017 : Introducción y lectura de datos (de CSV a EXCEL)
# --------------------------------------------------------------------------------------------------------------

## INTRODUCCIÓN

El objetvo de esta práctica es partir de un dataset de entrada y responder a ciertas preguntas relativas a esos datos de entrada a través de las BD siguientes :

   a) BD SQL   : 
         - PostgreSQL
   b) BD NoSQL :
         - Riak
         - Cassandra
         - MongoDB
         - Neoj4j

Para ello, llevaremos a cabo los siguientes pasos :

1. Selecionar un dataset de ENTRADA público.
2. Transformar y limpiar el dataset de ENTRADA (en formato CSV) en un dataset de SALIDA (en formato Excel).
3. Cargar en las BD especificadas todos los datos del dataset de SALIDA Excel generado en el punto anterior.
4. Aplicar un conjunto de preguntas a los datos cargados en esas BD (preguntas formuladas en el lenguaje propio de cada BD).


En este Notebook se llevará a cabo el paso 2 (transformación y limpieza del dataset de ENTRADA).




## Dataset de partida

Vamos a utilizar un dataset de las PLAYAS de ESPAÑA correspondiente a los datos obtenidos en el año 2017.


La fuente de datos original es la siguiente :

http://opendata.esri.es/datasets/84ddbc8cf4104a579d579f6441fcaa8a_0

con :

   + 3.511 playas
   + 80 atributos



## Formato del fichero de ENTRADA

El datset de entrada es un fichero en formato **CSV** y utiliza el carácter **","** cómo separador de columnas.


## Formato del fichero de SALIDA

A partir del fichero **CSV** de entrada se generará un fichero de SALIDA en formato **EXCEL** (con los datos del dataset limpiados y transformados según consideremos oportuno para nuestros posteriores cálculos).

## Preguntas

Vamos a intentar resolver las siguientes preguntas:
    
- **QUERY_1)** Las 10 playas más largas (incluído el nombre de la provincia)
- **QUERY_2)** Las 10 playas más largas de una cierta provincia (incluído el nombre de la provincia)
- **QUERY_3)** Las playas de una comunidad autónoma concreta (ordenadas por longitud)
- **QUERY_4)** Las 10 provincias que más metros de playa tienen
- **QUERY_5**) Metros de playa de una cierta Comunidad Autónoma agrupados por provincia
- **QUERY_6)** ¿ Qué Comunidad tiene más metros de playa ?
- **QUERY_7)** ¿ Qué provincia tiene más playas ... ?
   1. **QUERY_7.1**) ¿ Qué provincia tiene más playas ... en número ?
   2. **QUERY_7.2** ¿ Qué provincia tiene más playas ... con nombres diferentes ?
   3. **QUERY_7.3)** ¿ Qué provincia tiene más playas ... nudistas ?
   4. **QUERY_7.4)** ¿ Qué provincia tiene más playas ... con bandera azul ?

# A) Preprocesado de datos

Se realizará un preprocesado de datos cuyo objetico es limpiar y normalizar los datos para que se puedan utilizar más fácilmente por el resto de modelos de BD.  

El formato de salida será un fichero **Microsoft Excel** que contendrá los siguientes hojas (datasets):

   1. **HOJA_PROVINCIAS :** provincias costeras de España que tengan playas (con la Comunidad Autónoma a la que corresponde).
   2. **HOJA_PLAYAS** : playas de ESPAÑA y sus características a fecha de 2017.

## A.1) Importar librerías

In [1]:
import pandas as pd

## A.2) Leer la fuente de datos CSV en un DataFrame

In [2]:
df_csv = pd.read_csv("./data/playas_2017.csv", sep=',',  index_col=False )

In [3]:
# Verificamos que el df tiene las mismas dimensiones que el fichero CSV leído :
#    - Columnas : 80
#    - Filas    : 3511
df_csv.shape   

(3511, 80)

In [4]:
df_csv.head(3)

,X,Y,OBJECTID,Comunidad,Provincia,Isla,Código_IN,Núcleo_de_Población,Web_munici,Identifica,...,Dirección,Teléfono_,Distancia1,Composici,Fachada_Li,Espacio_pr,Espacio__1,Coordena_4,Coordena_5,URL_MAGRAM
0,-14.0487,28.5718,3001,Canarias,Las Palmas,Fuerteventura,35017,Puerto del Rosario,http://www.puertodelrosario.org,3007,...,"Ctra. del Aeropuerto, Km 1 (Puerto del Rosario)",928862000,37 km.,Grava / Arena,Dunas,No,,-14.0487,28.5718,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...
1,-14.0089,28.6729,3002,Canarias,Las Palmas,Fuerteventura,35014,La Oliva,http://www.laoliva.es,3008,...,"Ctra. del Aeropuerto, Km 1 (Puerto del Rosario)",928862000,40 km.,Grava / Arena,Dunas,No,,-14.0089,28.6729,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...
2,-14.0086,28.6771,3003,Canarias,Las Palmas,Fuerteventura,35014,La Oliva,http://www.laoliva.es,3009,...,"Ctra. del Aeropuerto, Km 1 (Puerto del Rosario)",928862000,"59,8 km.",Bolos / Grava / Arena,Montaña,No,,-14.0086,28.6771,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...


## A.3) Crear las PROVINCIAS

Creamos un DataFrame de **PROVINCIAS** (con su Comunidad Autónoma asociada) eliminado los registros duplicados.

In [5]:
df_provincias = df_csv[["Provincia",
                        "Comunidad"]].drop_duplicates()
df_provincias.head(5)

,Provincia,Comunidad
0,Las Palmas,Canarias
50,Almería,Andalucía
52,Barcelona,Cataluña/Catalunya
58,Castellón/Castelló,Comunitat Valenciana
62,Pontevedra,Galicia


In [6]:
df_provincias.shape

(24, 2)

Asignamos a cada provincia un ID único (a partir del nombre de la provincia) y lo añadimos al DataFrame de **PROVINCIAS**.

In [7]:
df_provincias = df_provincias.assign(id_provincia = df_provincias.Provincia.rank(method='min').astype(int))
df_provincias = df_provincias[['id_provincia', 'Provincia', 'Comunidad']]
df_provincias.sort_values('id_provincia')

,id_provincia,Provincia,Comunidad
116,1,A Coruña,Galicia
183,2,Alicante/Alacant,Comunitat Valenciana
50,3,Almería,Andalucía
131,4,Asturias,"Asturias, Principado de"
52,5,Barcelona,Cataluña/Catalunya
182,6,Bizkaia,País Vasco/Euskadi
180,7,Cantabria,Cantabria
58,8,Castellón/Castelló,Comunitat Valenciana
448,9,Ceuta,Ceuta
248,10,Cádiz,Andalucía


Insertamos en el dataset orginal (en el DataFrame obtenido al leer el fichero CSV fuente de los datos) el ID que acabamos de crear (id_provincia) .

In [8]:
df_merge = pd.merge(df_csv, df_provincias, on = ['Provincia', 'Comunidad'], how = 'inner')
df_merge.head(2)

,X,Y,OBJECTID,Comunidad,Provincia,Isla,Código_IN,Núcleo_de_Población,Web_munici,Identifica,...,Teléfono_,Distancia1,Composici,Fachada_Li,Espacio_pr,Espacio__1,Coordena_4,Coordena_5,URL_MAGRAM,id_provincia
0,-14.0487,28.5718,3001,Canarias,Las Palmas,Fuerteventura,35017,Puerto del Rosario,http://www.puertodelrosario.org,3007,...,928862000,37 km.,Grava / Arena,Dunas,No,,-14.0487,28.5718,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...,16
1,-14.0089,28.6729,3002,Canarias,Las Palmas,Fuerteventura,35014,La Oliva,http://www.laoliva.es,3008,...,928862000,40 km.,Grava / Arena,Dunas,No,,-14.0089,28.6729,http://sig.magrama.gob.es/93/ClienteWS/Guia-Pl...,16


Renombramos las columnas del DataFrame de **PROVINCIAS** para quitarnos acentos, mayúsculas y aclarar nombres.

In [9]:
df_provincias = df_provincias.rename(columns = {"Provincia" : "nom_provincia",     
                                                "Comunidad" : "comunidad"})
df_provincias.head(3)

,id_provincia,nom_provincia,comunidad
0,16,Las Palmas,Canarias
50,3,Almería,Andalucía
52,5,Barcelona,Cataluña/Catalunya


## A.4) Crear las PLAYAS

Seleccionamos las columnas que nos interesan para crear un DataFame de **PLAYAS**.

In [10]:
df_playas = df_merge[["id_provincia",
                      "OBJECTID", 
                      "Nombre", 
                      "Núcleo_de_Población",
                      "Longitud",
                      "Tipo_de_ar",
                      "Nudismo",
                      "Bandera_az"]]
df_playas.head(3)

,id_provincia,OBJECTID,Nombre,Núcleo_de_Población,Longitud,Tipo_de_ar,Nudismo,Bandera_az
0,16,3001,Janubio,Puerto del Rosario,230 metros,Dorada,Sí,No
1,16,3002,El Aljibe,La Oliva,1.210 metros,Dorada,Sí,No
2,16,3003,El Castillo,La Oliva,190 metros,Dorada,Sí,No


Renombramos las columnas del DataFrame de **PLAYAS** para quitarnos acentos, mayúsculas y aclarar nombres.

In [11]:
df_playas = df_playas.rename(columns = {"Nombre"              : "nom_playa", 
                                        "OBJECTID"            : "cod_playa", 
                                        "Núcleo_de_Población" : "localidad", 
                                        "Longitud"            : "longitud_txt", 
                                        "Tipo_de_ar"          : "arena", 
                                        "Nudismo"             : "nudista", 
                                        "Bandera_az"          : "bandera_azul"})
df_playas.head(3)

,id_provincia,cod_playa,nom_playa,localidad,longitud_txt,arena,nudista,bandera_azul
0,16,3001,Janubio,Puerto del Rosario,230 metros,Dorada,Sí,No
1,16,3002,El Aljibe,La Oliva,1.210 metros,Dorada,Sí,No
2,16,3003,El Castillo,La Oliva,190 metros,Dorada,Sí,No


In [12]:
df_playas.shape  #  Verifcamos que seguimos con los 3511 registros

(3511, 8)

Consideramos que una **clave única** de cada registro (playa) puede ser el par **(nom_playa, localidad)**, dado que en una misma localidad se supone que no puede haber 2 playas con el mismo nombre.
Añadimos al DataFrame de **PLAYAS** una **nueva columna** con la **concatenación de esos 2 campos** (clave_nom_playa).

In [13]:
df_playas.insert (4, "clave_nom_playa", df_playas["localidad"] +  " (" + 
                                        df_playas["nom_playa"] + ")")
df_playas.head(10)

,id_provincia,cod_playa,nom_playa,localidad,clave_nom_playa,longitud_txt,arena,nudista,bandera_azul
0,16,3001,Janubio,Puerto del Rosario,Puerto del Rosario (Janubio),230 metros,Dorada,Sí,No
1,16,3002,El Aljibe,La Oliva,La Oliva (El Aljibe),1.210 metros,Dorada,Sí,No
2,16,3003,El Castillo,La Oliva,La Oliva (El Castillo),190 metros,Dorada,Sí,No
3,16,3004,Marfolín,La Oliva,La Oliva (Marfolín),950 metros,Dorada,Sí,No
4,16,3005,Chica,La Oliva,La Oliva (Chica),200 metros,Blanca,No,No
5,16,3006,Las Playitas,Tuineje,Tuineje (Las Playitas),700 metros,Oscura,No,No
6,16,3007,Gran Tarajal,Tuineje,Tuineje (Gran Tarajal),560 metros,Dorada,No,No
7,16,3008,Giniginamar,Tuineje,Tuineje (Giniginamar),550 metros,Oscura,No,No
8,16,3009,El Embarcadero,Haría,Haría (El Embarcadero),180 metros,Dorada,Sí,No
9,16,3010,El Risco,Haría,Haría (El Risco),845 metros,Dorada,Sí,No


In [14]:
df_playas.insert (5, "longitud", int("0"))
df_playas.head(10)

,id_provincia,cod_playa,nom_playa,localidad,clave_nom_playa,longitud,longitud_txt,arena,nudista,bandera_azul
0,16,3001,Janubio,Puerto del Rosario,Puerto del Rosario (Janubio),0,230 metros,Dorada,Sí,No
1,16,3002,El Aljibe,La Oliva,La Oliva (El Aljibe),0,1.210 metros,Dorada,Sí,No
2,16,3003,El Castillo,La Oliva,La Oliva (El Castillo),0,190 metros,Dorada,Sí,No
3,16,3004,Marfolín,La Oliva,La Oliva (Marfolín),0,950 metros,Dorada,Sí,No
4,16,3005,Chica,La Oliva,La Oliva (Chica),0,200 metros,Blanca,No,No
5,16,3006,Las Playitas,Tuineje,Tuineje (Las Playitas),0,700 metros,Oscura,No,No
6,16,3007,Gran Tarajal,Tuineje,Tuineje (Gran Tarajal),0,560 metros,Dorada,No,No
7,16,3008,Giniginamar,Tuineje,Tuineje (Giniginamar),0,550 metros,Oscura,No,No
8,16,3009,El Embarcadero,Haría,Haría (El Embarcadero),0,180 metros,Dorada,Sí,No
9,16,3010,El Risco,Haría,Haría (El Risco),0,845 metros,Dorada,Sí,No


In [15]:
i=0
for texto in df_playas ["longitud_txt"]:
    texto_new = texto
    texto_new = texto_new.split(" ")[0]         # Eliminamos todo lo que sigue al número (el texto "metros")
    texto_new = texto_new.replace (".", "")     # Eliminamos los puntos (=> elimino los puntos de los miles)
    texto_new = texto_new.replace (" ", "")     # Eliminamos los espacios
    df_playas ["longitud"][i] = int (texto_new) # Cambiamos el tipo del campo de char a int
    i = i+1

In [16]:
df_playas.head(10)

,id_provincia,cod_playa,nom_playa,localidad,clave_nom_playa,longitud,longitud_txt,arena,nudista,bandera_azul
0,16,3001,Janubio,Puerto del Rosario,Puerto del Rosario (Janubio),230,230 metros,Dorada,Sí,No
1,16,3002,El Aljibe,La Oliva,La Oliva (El Aljibe),1210,1.210 metros,Dorada,Sí,No
2,16,3003,El Castillo,La Oliva,La Oliva (El Castillo),190,190 metros,Dorada,Sí,No
3,16,3004,Marfolín,La Oliva,La Oliva (Marfolín),950,950 metros,Dorada,Sí,No
4,16,3005,Chica,La Oliva,La Oliva (Chica),200,200 metros,Blanca,No,No
5,16,3006,Las Playitas,Tuineje,Tuineje (Las Playitas),700,700 metros,Oscura,No,No
6,16,3007,Gran Tarajal,Tuineje,Tuineje (Gran Tarajal),560,560 metros,Dorada,No,No
7,16,3008,Giniginamar,Tuineje,Tuineje (Giniginamar),550,550 metros,Oscura,No,No
8,16,3009,El Embarcadero,Haría,Haría (El Embarcadero),180,180 metros,Dorada,Sí,No
9,16,3010,El Risco,Haría,Haría (El Risco),845,845 metros,Dorada,Sí,No


In [17]:
df_playas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3511 entries, 0 to 3510
Data columns (total 10 columns):
id_provincia       3511 non-null int64
cod_playa          3511 non-null int64
nom_playa          3511 non-null object
localidad          3511 non-null object
clave_nom_playa    3511 non-null object
longitud           3511 non-null int64
longitud_txt       3511 non-null object
arena              3511 non-null object
nudista            3511 non-null object
bandera_azul       3511 non-null object
dtypes: int64(3), object(7)
memory usage: 461.7+ KB


# B) Creamos el fichero Excel con los datos finales

Grabamos el resultado en un fichero Excel donde cada dataset (**PROVINCIAS** y **PLAYAS**) estará en un hoja diferente.

In [18]:
writer = pd.ExcelWriter("./data/playas_2017.xlsx")
df_playas.to_excel    (writer, 'HOJA_PLAYAS')
df_provincias.to_excel(writer, 'HOJA_PROVINCIAS')
writer.save()

# FIN